In [1]:
import torch

import meshzoo

from torchfem.elements import linear_to_quadratic
from torchfem import Solid
from torchfem.materials import Isotropic

# Material model
material = Isotropic(E=1000.0, nu=0.3)

# 3D Cube with linear hexaeders

In [2]:
# Generate cube
points, cells = meshzoo.cube_hexa(
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
)
nodes = torch.tensor(points)
elements = torch.tensor(cells.tolist())

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1

# Create model
box = Solid(nodes, elements, forces, displacements, constraints, material.C())

# Solve
u, f = box.solve()

# Plot
box.plot(u=u, node_property={"Disp": u})

Widget(value='<iframe src="http://localhost:60984/index.html?ui=P_0x32e3f23e0_0&reconnect=auto" class="pyvista…

## 3D Cube with quadratic hexaeders

In [3]:
# Upgrade elements to quadratic
nodes, elements = linear_to_quadratic(nodes, elements)

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1

# Create model
box = Solid(nodes, elements, forces, displacements, constraints, material.C())

# Solve
u, f = box.solve()

# Plot
box.plot(u=u, node_property={"Disp": u})

Widget(value='<iframe src="http://localhost:60984/index.html?ui=P_0x106063700_1&reconnect=auto" class="pyvista…

# 3D Cube with linear hexaeders and inelastic (e.g. thermal) strain

In [4]:
# Reset constraints
displacements = torch.zeros_like(nodes)
constraints[:, :] = False
constraints[nodes[:, 0] == 0.0, 0] = True
constraints[nodes[:, 0] == 1.0, 0] = True
constraints[nodes[:, 1] == 0.5, 1] = True
constraints[nodes[:, 2] == 0.5, 2] = True

# Set inelastic strain (isotropic shrinkage)
s = -0.1
strains = torch.stack(len(elements) * [torch.tensor([s, s, s, 0.0, 0.0, 0.0])])

box = Solid(nodes, elements, forces, displacements, constraints, material.C(), strains)

In [5]:
u, f = box.solve()
box.plot(u=u, node_property={"Disp": u}, show_undeformed=True)

Widget(value='<iframe src="http://localhost:60984/index.html?ui=P_0x330d731c0_2&reconnect=auto" class="pyvista…

In [6]:
sigma = box.compute_stress(u)
print(f"Mean stress in x is {sigma[:,0].mean()}. It should be {-material.E() * s}.")

Mean stress in x is 99.99999237060547. It should be 100.0.


## 3D Cube with linear tetrahedrons

In [7]:
# Generate cube
points, cells = meshzoo.cube_tetra(
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
)
nodes = torch.tensor(points)
elements = torch.tensor(cells.tolist())

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1


box = Solid(nodes, elements, forces, displacements, constraints, material.C())

u, f = box.solve()

box.plot(u=u, node_property={"Disp": u})

Widget(value='<iframe src="http://localhost:60984/index.html?ui=P_0x330e934f0_3&reconnect=auto" class="pyvista…

## 3D Cube with quadratic tetrahedrons

In [8]:
nodes, elements = linear_to_quadratic(nodes, elements)

# Assign boundary conditions
forces = torch.zeros_like(nodes)
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True
constraints[nodes[:, 0] == 1.0, 0] = True
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] == 1.0, 0] = 0.1


box = Solid(nodes, elements, forces, displacements, constraints, material.C())

u, f = box.solve()

box.plot(u=u, node_property={"Disp": u})

Widget(value='<iframe src="http://localhost:60984/index.html?ui=P_0x330e93d00_4&reconnect=auto" class="pyvista…

# 3D Cube with tetrahedrons and inelastic (e.g. thermal) strain

In [9]:
# Reset constraints
displacements = torch.zeros_like(nodes)
constraints[:, :] = False
constraints[nodes[:, 0] == 0.0, 0] = True
constraints[nodes[:, 0] == 1.0, 0] = True
constraints[nodes[:, 1] == 0.5, 1] = True
constraints[nodes[:, 2] == 0.5, 2] = True

# Set inelastic strain (isotropic shrinkage)
s = -0.1
strains = torch.stack(len(elements) * [torch.tensor([s, s, s, 0.0, 0.0, 0.0])])

box = Solid(nodes, elements, forces, displacements, constraints, material.C(), strains)

In [10]:
u, f = box.solve()
box.plot(u=u, node_property={"Disp": u}, show_undeformed=True)

Widget(value='<iframe src="http://localhost:60984/index.html?ui=P_0x330e92b30_5&reconnect=auto" class="pyvista…

In [11]:
sigma = box.compute_stress(u)
print(f"Mean stress in x is {sigma[:,0].mean()}. It should be {-material.E() * s}.")

Mean stress in x is 99.99996948242188. It should be 100.0.
